In [ ]:
hsi_data=np.concatenate((vrf_refl,srf_refl), axis=2)
shape=hsi_data.shape
hsi_data=hsi_data.reshape(shape[-1], shape[0]* shape[1])

mask=~np.all(hsi_data  == 0 , axis=1)
X=hsi_data[mask].reshape(shape[0]* shape[1], shape[-1])

from sklearn.preprocessing import MinMaxScaler
n_components=10
pca=PCA(n_components=n_components)

X_transformed=pca.fit_transform(X)
variance_ratios=pca.explained_variance_ratio_

scaler = MinMaxScaler()
scaled=scaler.fit_transform(X_transformed)
mask = scaled == 0.0
masked_image = np.ma.masked_array(scaled, mask=mask)
p3=scaled[:,0:3]
p3=p3.reshape(shape[0],shape[1],3)
# imshow(p3, cmap='viridis')

pca10=scaled.reshape(shape[0],shape[1], n_components)

pca10=np.moveaxis(pca10,2,0)
prisma_pca_fp=f'../data/cairo/processed/prisma/primsa_pca.tif'

if os.path.exists(prisma_pca_fp) == False:
    # Save raster to file
    with r.open(
        prisma_pca_fp, 'w', 
        driver='GTiff', 
        height=pca10.shape[1], 
        width=pca10.shape[2], 
        count=n_components, 
        dtype=pca10.dtype, 
        crs='EPSG:4326', 
        transform=transform,
        nodata=np.nan

    ) as dst:
        dst.write(pca10)



In [ ]:
import rioxarray as rio
s2=rio.open_rasterio(S2_FP)
pca10=rio.open_rasterio(prisma_pca_fp)


s2=s2.rio.reproject(dst_crs='EPSG:4326')
s2=s2.sel(x=slice(x1,x2), y= slice(y2,y1))

pc10_match=pca10.rio.reproject_match(s2)

pc10_match.rio.to_raster(prisma_pca_fp.replace('.tif','_matched.tif'))